# Cats Vs Dogs Benchmark

In [3]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/lukewood/spectral-neural-nets && cd spectral-neural-nets && pip install .

### Data

In [5]:
import tensorflow as tf
NUM_CLASSES = 2
IMG_SIZE=(150, 150)
def makeThumb(img, size=(150,150)):
  return tf.keras.preprocessing.image.smart_resize(img, size)/255.0

In [6]:
import tensorflow_datasets as tfds
cats_vs_dogs = None
try:
  cats_vs_dogs = tfds.image_classification.cats_vs_dogs.CatsVsDogs()
except:
  cats_vs_dogs = tfds.image.cats_vs_dogs.CatsVsDogs()
cats_vs_dogs.download_and_prepare()
dataset = cats_vs_dogs.as_dataset(shuffle_files=False)

In [7]:
def to_numpy(ds, size=None):
  X = []
  y = []
  csize=0
  for datapoint in ds:
    if size != None and csize > size:
      return X, y
    X.append(makeThumb(datapoint['image'].numpy(), size=IMG_SIZE))
    y.append(tf.keras.utils.to_categorical(datapoint['label'].numpy(), NUM_CLASSES))
    csize+=1
  return X, y

def to_gen(X, y, batch_size=32):
  idx = 0
  while idx < len(X) and idx < len(y):
    Xs = []
    ys = []
    for _ in range(batch_size):
      if idx >= len(X) or idx >= len(y):
        break
      Xs.append(X[idx])
      ys.append(y[idx])
      idx+=1
    yield np.array(Xs), np.array(ys)

In [8]:
X, y = to_numpy(dataset['train'])

## Image Augmentation

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=30,
    zoom_range=[0.5, 1.0],
    brightness_range=(0.85, 1.15),
    horizontal_flip=True
)

## Evaluation

In [10]:
from sklearn.metrics import precision_score, confusion_matrix, accuracy_score
def accuracy_for_model(model, X_test, y_test):
  y_pred = model.predict(to_gen(X_test, y_test))
  y_pred = np.argmax(y_pred, axis=1)
  y_test_max = np.argmax(np.array(y_test).squeeze(), axis=1)

  return accuracy_score(y_test_max, y_pred)

# Models

In [15]:
from tensorflow.keras.layers import Input, Dense, Flatten, Activation
from tensorflow.keras import Model
from spectral_neural_nets.layers import LinearKernelFourierConvolution, fft_layer, from_complex

def linear_fourier_conv_block(x, filters=6, order=1):
  x = LinearKernelFourierConvolution(filters, order=order)(x)
  x = Activation('relu')(x)
  return x

def create_linear_model(filter_blocks = [6, 6, 6, 6], order=1):
  inputs = Input(shape=(150, 150, 3))
  x = fft_layer(inputs)
  x = from_complex(x)
  for f in filter_blocks:
    x = linear_fourier_conv_block(x, filters=f, order=order)
  x = Flatten()(x)
  preds = Dense(2)(x)

  model = Model(inputs, preds)
  optimizer = tf.optimizers.Adam()
  return model
model=create_linear_model()

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Activation, Dense

def convblock(x, filters=6):
  x = Conv2D(filters, 3, padding='same', use_bias=True)(x)
  x = Activation('relu')(x)
  return x

def create_spatial_model(filter_blocks = [6, 6, 6, 6]):
  inputs = Input(shape=IMG_SIZE + (3,))
  x = inputs
  for f in filter_blocks:
    x = convblock(x, filters=f)
  x = Flatten()(x)
  preds = Dense(NUM_CLASSES)(x)

  model = Model(inputs=inputs, outputs=preds)
  return model

model = create_spatial_model()

In [13]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Activation, Dense
from spectral_neural_nets.layers import FourierDomainConv2D, fft_layer, from_complex

def fourier_block(x, filters=6):
  x = FourierDomainConv2D(filters)(x)
  x = Activation('relu')(x)
  return x

def create_fourier_model(filter_blocks = [6, 6, 6, 6]):
  inputs = Input(shape=(150, 150, 3))
  x = fft_layer(inputs)
  x = from_complex(x)
  for f in filter_blocks:
    x = fourier_block(x, filters=f)
  x = Flatten()(x)
  preds = Dense(NUM_CLASSES)(x)

  model = Model(inputs=inputs, outputs=preds)
  return model

model = create_fourier_model()

In [14]:
from tensorflow.keras.layers import Input, Dense, Flatten, Activation
from tensorflow.keras import Model
from spectral_neural_nets.layers import Gaussian2DFourierLayer, fft_layer, from_complex

def gaussian_block(x, filters=6, order=1):
  x = Gaussian2DFourierLayer(filters)(x)
  x = Activation('relu')(x)
  return x

def create_gaussian_model(filter_blocks = [6, 6, 12, 12, 24, 24], order=1):
  inputs = Input(shape=(150, 150, 3))
  x = fft_layer(inputs)
  x = from_complex(x)
  for f in filter_blocks:
    x = gaussian_block(x, filters=f, order=order)
  x = Flatten()(x)
  preds = Dense(2)(x)

  model = Model(inputs, preds)
  optimizer = tf.optimizers.Adam()
  return model
model=create_gaussian_model()

# Testing

In [1]:
import sklearn.model_selection
import numpy as np
from IPython.display import clear_output

trials=20
training_epochs=3

results = {}

models_to_test = [
    ("spatial", create_spatial_model),
    ("fourier", create_fourier_model),
    ("parametric=linear", create_linear_model),
    ("parametric=gaussian", create_gaussian_model)
]

for name, _ in models_to_test:
  results[name] = []

for _ in range(trials):
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.33)
  datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
  )
  X_train = np.array(X_train)
  y_train = np.array(y_train)
  datagen.fit(X_train)
  for model_name, model_creation_fn in models_to_test:
    model = model_creation_fn()
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
    history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
          steps_per_epoch=len(X_train) / 64, epochs=training_epochs)
    results[model_name].append(accuracy_for_model(model, X_test, y_test))
    clear_output(wait=True)
    print("results=", results)
  clear_output(wait=True)
  print("results=", results)

NameError: name 'create_spatial_model' is not defined

In [17]:
import seaborn as sns
import pandas as pd

data = []

for model_name, model_accuracies in results.items():
    data.append({'model_type': model_name, 'accuracy': model_accuracies})
df = pd.DataFrame(data=data)
ax = sns.violinplot(x="model type", y="accuracy", data=df)

ValueError: Could not interpret input 'model type'

In [ ]:
from statistics import mean, median, stdev
from tabulate import tabulate

headers=["Network", "Mean Acc", "Median Acc", "Stdev"]

def gather_statistics(network,scores):
  return {
      "network": network,
      "mean": mean(scores),
      "max": max(scores),
      "median": median(scores),
      "stdev": stdev(scores)
  }

statistics = []
for model_name, model_accuracies in results.items()
    statistics.append(gather_statistics(model_name, model_accuracies))

df = pd.DataFrame(data=statistics)
df